In [9]:
import os
import re
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt

# --- GT mapping for conditions ---
CONDITION_GT = {
    'A': {'name': "Healthy rest",              'RR': 14, 'HR': 80},
    'B': {'name': "Healthy sleeping",          'RR': 12, 'HR': 60},
    'C': {'name': "Healthy agitated",          'RR': 20, 'HR': 110},
    'D': {'name': "Apnea",                     'RR': 0,  'HR': 80},
    'E': {'name': "Asthma",                    'RR': 22, 'HR': 90},
    'F': {'name': "Asthma attack",             'RR': 33, 'HR': 130},
    'G': {'name': "Pneumothorax",              'RR': 30, 'HR': 120},
    'H': {'name': "Bradypnea",                 'RR': 6,  'HR': 50},
    'I': {'name': "Cardiac arrest",            'RR': 16, 'HR': 160},
    'J': {'name': "ACS Chest Pain",            'RR': 18, 'HR': 104},
    'K': {'name': "Tachypnea/Hyperventilation",'RR': 30, 'HR': 125},
}

# --- Antenna coordinates (mm) ---
RX_POS = np.array([
    [0.000, +1.800, 0.0],   # Rx0
    [+1.800, 0.000, 0.0],   # Rx1
    [0.000, -1.800, 0.0],   # Rx2
    [-1.800, 0.000, 0.0]    # Rx3
])
TX_POS = np.array([
    [+0.900, +0.900, 0.0],  # Tx0
    [-0.900, +0.900, 0.0],  # Tx1
    [0.000, -0.900, 0.0]    # Tx2
])

def parse_iwr6843_filename(filename):
    basename = os.path.basename(filename)
    pattern = (
        r'(?P<id>\d+)_.*_Man\d+_'
        r'Cond(?P<cond_a>[A-Z])_Cond(?P<cond_b>[A-Z])_Cond(?P<cond_c>[A-Z])_'
        r'ADC(?P<adc>\d+)_Chirp(?P<chirp>\d+)_SR(?P<sr>\d+)_RE(?P<re>\d+)_FR(?P<fr>\d+)_Gain(?P<gain>\d+)_FS(?P<fs>\d+)_IWR'
    )
    m = re.match(pattern, basename)
    if m:
        params = {k: (int(v) if v.isdigit() else v) for k, v in m.groupdict().items()}
        return params
    else:
        raise ValueError(f"Filename {basename} does not match expected pattern")

def decode_iwr6843_data(filepath, params, num_rx=4, header_bytes=0):
    num_adc_samples = int(params['adc'])
    num_chirps      = int(params['chirp'])

    with open(filepath, 'rb') as f:
        f.seek(header_bytes)
        raw = np.fromfile(f, dtype=np.int16)

    if raw.size % 2 != 0:
        raw = raw[:-1]
    raw = raw.reshape(-1, 2)
    complex_data = raw[:, 0].astype(np.float32) + 1j * raw[:, 1].astype(np.float32)

    samp_pf    = num_rx * num_chirps * num_adc_samples
    num_frames = complex_data.size // samp_pf
    if num_frames == 0:
        print(f"Warning: No frames found for {filepath}. Check ADC/Chirp parameters.")
        return None

    complex_data = complex_data[: num_frames * samp_pf]
    adc = np.empty((num_frames, num_rx, num_chirps, num_adc_samples), dtype=np.complex64)
    for fr in range(num_frames):
        base = fr * samp_pf
        for rx in range(num_rx):
            st = base + rx * (num_chirps * num_adc_samples)
            en = st + (num_chirps * num_adc_samples)
            adc[fr, rx] = complex_data[st:en].reshape(num_chirps, num_adc_samples)
    return adc

def range_angle_processing(adc_cube, params):
    # adc_cube shape: (num_frames, num_rx, num_chirps, num_adc_samples)
    # 1. Range FFT
    num_frames, num_rx, num_chirps, num_adc_samples = adc_cube.shape
    range_fft = np.fft.fft(adc_cube, axis=-1)
    # 2. Doppler FFT
    rd_cube = np.fft.fftshift(np.fft.fft(range_fft, axis=2), axes=2)  # shape: (frames, rx, chirps, range_bins)
    # For spatial separation (AoA):
    # Average over frames and chirps to get range-antenna matrix
    # For simplicity, use the mean across frames and chirps
    ra_matrix = np.mean(rd_cube, axis=(0,2))  # shape: (rx, range_bins)
    # 3. AoA Estimation: MUSIC/Capon or simple beamforming
    # Here we use simple beamforming for demonstration, MUSIC/Capon can be added for more accuracy
    # For each range bin (bed), estimate AoA using antenna array
    num_range_bins = ra_matrix.shape[1]
    angles = np.linspace(-60, 60, 181) # degrees
    separated_signals = []
    for rng_idx in [int(num_range_bins/4), int(num_range_bins/2), int(3*num_range_bins/4)]:  # Assume 3 beds
        sig = ra_matrix[:, rng_idx]  # (rx,)
        # Beamforming: steering vector
        bed_signals = []
        for theta in [-30, 0, +30]:  # Example AoAs for 3 beds
            theta_rad = np.deg2rad(theta)
            steering = np.exp(1j * 2 * np.pi * RX_POS[:,1] * np.sin(theta_rad) / 0.004)  # lambda=4mm for 77GHz
            bed_signal = np.dot(sig, steering)
            bed_signals.append(bed_signal)
        separated_signals.append(bed_signals)
    # separated_signals shape: (range_bins, beds)
    return separated_signals  # list of [bed1, bed2, bed3] signals

def bandpass_filter(data, lowcut, highcut, fs, order=4):
    sos = signal.butter(order, [lowcut, highcut], btype='band', fs=fs, output='sos')
    return signal.sosfilt(sos, data)

def vital_sign_extraction(bed_signal, fs=20):
    phase = np.unwrap(np.angle(bed_signal))
    # Respiration: 0.2–0.5 Hz
    rr_filtered = bandpass_filter(phase, 0.2, 0.5, fs)
    rr_fft = np.abs(np.fft.fft(rr_filtered))
    rr_freq = np.argmax(rr_fft[:len(rr_fft)//2]) * fs / len(rr_fft)
    RR = int(rr_freq * 60)  # breaths per minute

    # Heartbeat: 0.8–2.0 Hz
    hr_filtered = bandpass_filter(phase, 0.8, 2.0, fs)
    hr_fft = np.abs(np.fft.fft(hr_filtered))
    hr_freq = np.argmax(hr_fft[:len(hr_fft)//2]) * fs / len(hr_fft)
    HR = int(hr_freq * 60)  # beats per minute
    return RR, HR

def process_file(filepath):
    params = parse_iwr6843_filename(filepath)
    adc_cube = decode_iwr6843_data(filepath, params)
    separated = range_angle_processing(adc_cube, params)
    results = []
    for bed_idx, bed_signal in enumerate(separated):
        RR, HR = vital_sign_extraction(bed_signal)
        cond = params[f'cond_{chr(ord("a")+bed_idx)}']
        gt = CONDITION_GT[cond]
        results.append({
            'file': filepath,
            'bed': bed_idx,
            'condition': gt['name'],
            'RR_gt': gt['RR'],
            'HR_gt': gt['HR'],
            'RR_est': RR,
            'HR_est': HR,
            'RR_err': RR - gt['RR'],
            'HR_err': HR - gt['HR']
        })
    return results

def process_folder(folder):
    all_results = []
    for fname in os.listdir(folder):
        if not fname.endswith('.bin'):
            continue
        filepath = os.path.join(folder, fname)
        try:
            results = process_file(filepath)
            all_results.extend(results)
        except Exception as e:
            print(f"Error processing {fname}: {e}")
    return all_results

def save_results_csv(results, filename='vitals_results.csv'):
    import csv
    if not results:
        print('No results to save.')
        return
    keys = results[0].keys()
    with open(filename, 'w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=keys)
        writer.writeheader()
        writer.writerows(results)
    print(f'Results saved to {filename}')

# --- Example usage ---
if __name__ == "__main__":
    folder = "/home/jupyter-karla/paper_MIMIC/plots/SubsetA"
    results = process_folder(folder)
    save_results_csv(results)
    for r in results:
        print(r)

Error processing 379_TopFront_4m_0deg_Man3_CondF_CondL_CondE_ADC64_Chirp32_SR1000_RE80_FR24_Gain30_FS30_IWR.bin: 'L'
Error processing 385_TopFront_4m_0deg_Man3_CondF_CondC_CondL_ADC64_Chirp32_SR1000_RE80_FR60_Gain40_FS30_IWR.bin: 'L'
Error processing 387_TopFront_4m_0deg_Man3_CondH_CondL_CondC_ADC64_Chirp32_SR1000_RE80_FR60_Gain30_FS30_IWR.bin: 'L'
Results saved to vitals_results.csv
{'file': '/home/jupyter-karla/paper_MIMIC/plots/SubsetA/373_TopFront_4m_0deg_Man3_CondJ_CondB_CondE_ADC64_Chirp32_SR1000_RE60_FR40_Gain40_FS30_IWR.bin', 'bed': 0, 'condition': 'ACS Chest Pain', 'RR_gt': 18, 'HR_gt': 104, 'RR_est': 0, 'HR_est': 0, 'RR_err': -18, 'HR_err': -104}
{'file': '/home/jupyter-karla/paper_MIMIC/plots/SubsetA/373_TopFront_4m_0deg_Man3_CondJ_CondB_CondE_ADC64_Chirp32_SR1000_RE60_FR40_Gain40_FS30_IWR.bin', 'bed': 1, 'condition': 'Healthy sleeping', 'RR_gt': 12, 'HR_gt': 60, 'RR_est': 0, 'HR_est': 0, 'RR_err': -12, 'HR_err': -60}
{'file': '/home/jupyter-karla/paper_MIMIC/plots/SubsetA/3